In [13]:
from datasets import load_dataset
import os
import json 
livecodebench = load_dataset("livecodebench/code_generation_lite", version_tag="release_v5", trust_remote_code=True, split="test")

print(livecodebench)



Dataset({
    features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
    num_rows: 880
})


In [14]:
from datetime import datetime

# deepseek paper uses lcb from 2024-08 to 2025-01
def is_date_in_range(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S")
    
    # Define the start and end range
    start_date = datetime(2024, 8, 1)  # August 1, 2024
    end_date = datetime(2025, 2, 1)    # February 1, 2025

    # Check if the date is within the range
    return start_date <= date_obj < end_date

In [15]:

dataset = []
for entry in livecodebench:
    public_test_cases = entry["public_test_cases"]
    if isinstance(public_test_cases, str):
        try:
            public_test_cases = json.loads(public_test_cases)
        except json.JSONDecodeError as e:
            print(f"code reward Json Error parsing livecodebench: {e}")
            continue 
    if len(public_test_cases) == 0:
        #this does not have public_test_case
        continue 
    new_entry = {
        "problem": entry["question_content"],
        "starter_code": entry["starter_code"],
    }
    new_entry["public_test_cases"] = entry["public_test_cases"]
    dataset.append(new_entry)

print(f'len(dataset): {len(dataset)}')

output_dir = os.path.abspath("../../test/code")
output_file = os.path.join(output_dir, "livecodebench.json")

with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)


len(dataset): 879
